# Preparation

Load necessary modules and initialize relavent devices

In [1]:
%run seis_idd/tomo_init.py

/Users/chenzhang/opt/anaconda3/envs/bluesky/lib/python3.7/site-packages/databroker/_core.py:962: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)
/Users/chenzhang/opt/anaconda3/envs/bluesky/lib/python3.7/site-packages/epics/pv.py:110: UserWarning: The `context` kwarg for epics.get_pv() is deprecated. New PVs will _not_ be created in the requested context.
  'The `context` kwarg for epics.get_pv() is deprecated. New PVs '



🙊: Use init_tomo() to toggle between
    debug :     development mode, no hardware connection
    dryrun:     testing mode, connect to hardware, no beam
    production: ready for experiment



You can check the __prefined variables and functions__ in the following two dictionaries:

```
keywords_vars = {}  # {name: short description}
keywords_func = {}  # {name: short descciption}
```

or the corresponding functions:

```
list_predefined_vars()
list_predefined_func()
```

In [6]:
print('var:')
list_predefined_vars()

var:


name,description
HOSTNAME,host name
USERNAME,PI/user name
metadata_db,Default metadata handler
RE,Default RunEngine instance
A_shutter,shutter instance
suspend_A_shutter,no scans until A_shutter is open
tomostage,sim/real tomo stage
preci,rotation control
samX,tomo stage x-translation
ksamx,?


In [7]:
print('func:')
list_predefined_func()

func:


name,description
get_runengine,Get a bluesky RunEngine
load_config,load yaml to a dict
get_shutter,Return a connection to a sim/real shutter
get_motors,Return a connection to sim/real tomostage motor
get_fly_motor,Return a connection to fly IOC control
get_detector,Return a connection to the detector
instrument_in_use,"instrument status, manual set on IOC"
hutch_light_on,Hutch lighting status
init_tomo,(Re)-initialized all devices with given mode


__Fill__ out the __experiment details__ below for the metadata handler (MongoDB)

In [3]:
RE.md['beamline_id']      = 'APS 6-ID-D'
RE.md['proposal_id']      = 'internal test'
RE.md['pid']              = os.getpid()
RE.md['login_id']         = USERNAME + '@' + HOSTNAME
RE.md['BLUESKY_VERSION']  = bluesky.__version__
RE.md['OPHYD_VERSION']    = ophyd.__version__
RE.md['apstools_VERSION'] = apstools.__version__
RE.md['SESSION_STARTED']  = datetime.isoformat(datetime.now(), " ")

# Initialization

Perform control initialization for all related hardware.

In [ ]:
init_tomo(mode='debug')

Install global __suspenders__ as a safeguard for this experiment

In [ ]:
import apstools.devices      as APS_devices
from   bluesky.suspenders import SuspendFloor

aps = APS_devices.ApsMachineParametersDevice(name="APS")

# monitor ring current
suspend_APS_current = SuspendFloor(aps.current, 2, resume_thresh=10)
RE.install_suspender(suspend_APS_current)

# monitor shutter status
# NOTE: do not install right away since we need to close the shutter to collect dark field
suspend_A_shutter = SuspendFloor(A_shutter.pss_state, 1)


Import the __necessary__ predefined plan stubs

In [ ]:
from seis_idd.tomo_plans import tomo_scan

scan_cfg = 'seis_idd/config/tomo_scan_template.yml'

Make sure check the scan with _summarize_plan_ before running

In [ ]:
from bluesky.simulators import summarize_plan

In [ ]:
summarize_plan(tomo_scan(det, tomostage, A_shutter, suspend_A_shutter, scan_cfg))